In [1]:
from logging import getLogger
from datetime import datetime
import pandas as pd
from btc_functions.logging.logger_config import setup_logger
import os
from dotenv import load_dotenv
import btc_functions.database.mysql as db_functions

In [15]:
def reverse_timestamp(df: pd.DataFrame, col1: str, col2: str = None) -> pd.DataFrame:
    """Attention, la fonction traite max 2 colonnes

    Args:
        df (pd.DataFrame): df
        col1 (str): nom de la col à transformer
        col2 (str, optional): nom de la col2 à transformer. Defaults to None.

    Returns:
        pd.DataFrame: df
    """
    cols = [col1] if col2 is None else [col1, col2]
    df[cols] = df[cols].map(lambda x: datetime.fromtimestamp(x / 1000))
    return df


def fetch_and_process_table(table_name, col1, col2 = None) -> pd.DataFrame:
    """Récupère une table SQL, applique reverse_timestamp sur les cols BIGINT spécifiées
       et retourne le DataFrame.

    Args:
        engine (connection): engine pymysql
        table_name (sql_table): nom de la table cible (type BIGINT / timestamp)
        col1 (table_column): table à transcrire
        col2 (table_column2, optional): 2e colonne à transcrire. Defaults to None.
    """
    load_dotenv("/home/sanou/BTC_app/env/private.env")
    engine = db_functions.create_connection()
    query = f"SELECT * FROM {table_name}"

    df = pd.read_sql_query(query, engine)
    df = reverse_timestamp(df, col1, col2)
    
    # logger.info(f"Table {table_name} traitée avec succès.")
    return df

In [ ]:
load_dotenv("/home/sanou/BTC_app/env/private.env")
table_klines = "klines"
table_24 = "ticket24h"
tables = ["klines", "ticket24h"]

engine = db_functions.create_connection()

for table in tables:
    query = f"SELECT * FROM {table}"
    df = pd.read_sql_query(query, engine)
    bigint_cols = df.select_dtypes(include=['int64']).columns
    
    for col in bigint_cols:
        df = reverse_timestamp(df, bigint_cols)
        print(f'{col} traitée.')
        print(df.head())


table_klines = "klines"
col_kline1 = "kline_open_time"
col_kline2 = "kline_close_time"
table_24 = "ticket24h"
col_ticket1 = "openTime"
col_ticket2 = "closeTime"

query_klines = f"SELECT * FROM {table_klines}"
query_ticket = f"SELECT * FROM {table_24}"
df_klines = pd.read_sql_query(query_klines, engine)
df_24 = pd.read_sql_query(query_ticket, engine)

df_klines = reverse_timestamp(df_klines, col_kline1, col_kline2)
df_24 = reverse_timestamp(df_24, col_ticket1, col_ticket2)


print(df_klines.head(), df_24.head())

In [20]:
df_klines = fetch_and_process_table("klines", "kline_open_time", "kline_close_time")
df_24 = fetch_and_process_table("ticket24h", "openTime", "closeTime")

2024-11-23 02:02:13,282 - btc_functions.database.mysql - INFO - Tentative de connexion à localhost:3306 en tant que sanou
2024-11-23 02:02:13,284 - btc_functions.database.mysql - INFO - Connexion à MySQL réussie avec SQLAlchemy.
2024-11-23 02:02:13,361 - btc_functions.database.mysql - INFO - Tentative de connexion à localhost:3306 en tant que sanou
2024-11-23 02:02:13,361 - btc_functions.database.mysql - INFO - Connexion à MySQL réussie avec SQLAlchemy.


In [21]:
# df_klines
df_klines.head()

,kline_open_time,open_price,high_price,low_price,close_price,volume,kline_close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
0,2024-11-03 00:00:00,69434.00,69446.00,69187.06,69200.00,127.15359,2024-11-03 00:04:59.999,8.813560e+06,11995,21.10844,1.462531e+06
1,2024-11-03 00:05:00,69199.99,69290.00,69190.00,69289.99,26.09395,2024-11-03 00:09:59.999,1.806698e+06,7712,11.98186,8.295812e+05
2,2024-11-03 00:10:00,69289.99,69290.00,69224.00,69278.00,38.03960,2024-11-03 00:14:59.999,2.634169e+06,4769,20.39812,1.412487e+06
3,2024-11-03 00:15:00,69277.99,69293.99,69168.62,69168.63,31.82851,2024-11-03 00:19:59.999,2.203404e+06,4641,4.15051,2.874385e+05
4,2024-11-03 00:20:00,69168.62,69224.00,69168.62,69189.99,22.12902,2024-11-03 00:24:59.999,1.531406e+06,5483,10.88243,7.530376e+05


In [24]:
# df_24
df_24.head()

df_klines.to_csv("/home/sanou/BTC_app/1_python_scripts/test_kline.csv")
df_24.to_csv("/home/sanou/BTC_app/1_python_scripts/test_df24.csv")

In [10]:
from datetime import datetime
import requests
import json
import os 


today = datetime.utcnow()
start_of_month = today.replace(day=1)
days = (today - start_of_month).days + 2

url = "https://api.binance.com"
endpoint = "/api/v3/ticker/tradingDay"
params = {"symbol": "BTCUSDT"}
file = "prices_BTC_daily"

response = requests.get(url + endpoint, params=params)
if response.status_code == 200:
    data = response.json()

raw_data = "/home/sanou/BTC_app/data/1_raw/"
os.makedirs(raw_data, exist_ok=True)
file_path = f"/home/sanou/BTC_app/data/1_raw/{file}_{days}.json"
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, "w") as f:
    json.dump(data, f, indent=4)
print(start_of_month, days)

/tmp/ipykernel_27850/1256362074.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.utcnow()


2024-11-01 01:41:18.243220 24
